In [3]:
from pyspark.ml.feature import VectorAssembler,BucketedRandomProjectionLSHModel
from pyspark.sql.functions import radians, cos, sin, col,to_timestamp,lit,current_timestamp
loaded_model = BucketedRandomProjectionLSHModel.load("abfss://b3979a08-4bbe-4d35-b864-4da7d2c8b2b4@onelake.dfs.fabric.microsoft.com/608b9b1d-a537-4410-abb1-240bd8dc87ff/Files/NNmodel")

from pyspark.ml.linalg import Vectors


# 3. Convert (ra, dec, distance) → Cartesian (x, y, z)
df_coordinates = spark.read.format("delta").table("stg_coordinates")
#4. Assemble into a single feature vector column
assembler = VectorAssembler(inputCols=["x","y","z"], outputCol="features")
df_vec = assembler.transform(df_coordinates).select("source_id","features")

df_hashed = loaded_model.transform(df_vec)


query_feat = Vectors.dense([x, y, z])
print(query_feat)
# 8. Perform approximate nearest-neighbor search
nbrs = loaded_model.approxNearestNeighbors(
    dataset=df_hashed,
    key=query_feat,
    numNearestNeighbors=5,    # return top-5
    distCol="euclidean_dist"  # name of distance column
)
nbrs=nbrs.withColumn("id",lit(id)) \
.withColumn("timestamp",lit(current_timestamp())) \
.withColumn("x-coordinate",lit(x)) \
.withColumn("y-coordinate",lit(y)) \
.withColumn("z-coordinate",lit(z))

# 9. Display the results
selected_cols = ["id","source_id", "timestamp", "x-coordinate", "y-coordinate", "z-coordinate","euclidean_dist"]
nbrs.select(*selected_cols).write.format("delta").mode("append").saveAsTable("fsi_results")


StatementMeta(, 13b7fd27-3a7b-49df-b014-4210b6d18aac, 5, Finished, Available, Finished)

[2.0,3.0,3.0]


PySparkTypeError: [NOT_COLUMN] Argument `col` should be a Column, got int.